In [1]:
import pandas as pd
import sqlite3

In [2]:
connect = sqlite3.connect('../../datasets/checking-logs.sqlite')

shecma of the table test

In [3]:
pd.io.sql.read_sql('PRAGMA table_info(test);',connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


first 10 rows

In [4]:
pd.io.sql.read_sql('SELECT * FROM test LIMIT 10;', connect)

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


minimum value of the delta between the first commit of the user and the deadline of the corresponding lab

In [5]:
query = """
SELECT t.uid, 
       MIN(CAST((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INT)) AS min
FROM test AS t
JOIN deadlines AS dl ON t.labname = dl.labs
WHERE t.labname != 'project1'
"""
df_min = pd.io.sql.read_sql(query, connect)
df_min

,uid,min
0,user_30,-202


In [6]:
query = """
SELECT uid, MAX(delta) as max
FROM (
    SELECT uid, cast((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) *24 as integer) AS delta
    FROM test as t
    JOIN deadlines as dl ON t.labname = dl.labs
    WHERE t.labname != 'project1'
)
"""
df_max = pd.io.sql.read_sql(query, connect)
df_max

,uid,max
0,user_25,-2


In [7]:
query = """
SELECT AVG(calc) AS avg
FROM (
    SELECT CAST(
        (julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24
        AS INTEGER
    ) AS calc
    FROM test AS t
    JOIN deadlines AS dl ON t.labname = dl.labs
    WHERE t.labname != 'project1'
)
"""
df_avg = pd.io.sql.read_sql(query, connect)
df_avg

,avg
0,-89.125


In [8]:
query = """
WITH user_stats AS (
    SELECT 
        t.uid,
        -- Правильный расчет средней разницы времени в часах
        ROUND(AVG((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24), 2) AS avg_diff,
        
        -- Подсчет всех просмотров
        (SELECT COUNT(*) FROM pageviews p WHERE p.uid = t.uid) AS pageviews
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != 'project1'
    GROUP BY t.uid
    HAVING pageviews > 0
)
SELECT * FROM user_stats
"""

views_diff = pd.read_sql(query, connect)

views_diff

,uid,avg_diff,pageviews
0,user_1,-65.12,28
1,user_10,-75.24,89
2,user_14,-159.57,143
3,user_17,-62.21,47
4,user_18,-6.37,3
5,user_19,-99.44,16
6,user_21,-96.11,10
7,user_25,-93.47,179
8,user_28,-86.79,149
9,user_3,-105.74,317


In [9]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.279132
pageviews,-0.279132,1.000000


In [10]:
connect.close()